This is my submission for the task Tagging System of Questions using Transfer Learning. We use a kind of brute force method, using TF-IDF to predict the best possible tags for a question.


First, load the required libraries. 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk # natural language processing
import re # regular expression
from bs4 import BeautifulSoup #scraping HTML
from nltk.corpus import stopwords
import seaborn as sns # visualization
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from string import punctuation




# nltk workspace

stop = set(stopwords.words('english'))

biology.csv
cooking.csv
crypto.csv
diy.csv
robotics.csv
sample_submission.csv
test.csv
travel.csv



In [2]:
# read in test data sets
test = pd.read_csv("test.csv")

# **Data Cleaning**

The content column contains some HTML tags. We clean our data to remove these tags, convert all to small case letters and remove stopwords.

In [5]:
def parse_content(text):
    """
        text: a string
        
        return: modified initial string
    """
    #text = # lowercase text
    text =text.lower()
    #text = # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)
    #text = # delete symbols which are in BAD_SYMBOLS_RE from text
    text =  re.sub(BAD_SYMBOLS_RE, '', text)
	text = re.sub(r"\b[a-zA-Z]\b", "", text)
	text = re.sub("\d+", "", text)

    #text = # delete stopwords from text
    token_word=word_tokenize(text)
    filtered_sentence = [w for w in token_word if not w in STOPWORDS] # filtered_sentence contain all words that are not in stopwords dictionary
    lenght_of_string=len(filtered_sentence)
    text_new=""
    for w in filtered_sentence:
        if w!=filtered_sentence[lenght_of_string-1]:
             text_new=text_new+w+" " # when w is not the last word so separate by whitespace
        else:
            text_new=text_new+w
            
    text = text_new
    return text

In [6]:
# Apply parse_content onto dataframe.
raw = pd.concat([raw,raw['content'].apply(parse_content)],axis = 1)
raw = pd.concat([raw,raw['title'].apply(parse_content)],axis = 1)

We do further processing on our text by selecting nouns which match a certain patter, and converting the entire document into tokens of words.

In [14]:
#tokenize and tag texts. 
lemmatizer = nltk.stem.WordNetLemmatizer()
raw['all_text'] = raw['title'] + raw['content']
raw['text_token'] = raw['all_text'].apply(nltk.word_tokenize)
raw['text_token'] = raw['text_token'].apply(lambda x:[lemmatizer.lemmatize(t) for t in x])
raw['text_pos'] = raw['text_token'].apply(nltk.pos_tag)
raw['text_nouns'] = raw['text_pos'].apply(lambda x: [pair[0] for pair in x if pair[1] in ("NN","NNS","JJ")])

In [15]:
raw['text_bigram'] = raw['text_pos'].apply(nltk.bigrams)
raw['text_bigram'] = raw['text_bigram'].apply(list)

In [18]:
def findPair(l):
    result = []
    for pair in l:
        if pair[1][1] in ('NN','NNS') and pair[0][1] in ('NN','NNS','JJ'):
            result.append(pair[0][0]+" "+pair[1][0])
    return result
raw['word_pair'] = raw['text_bigram'].apply(findPair)

Now, lets make the actual predictions.

In [19]:
mydoclist = raw['text_nouns'].apply(" ".join).tolist()
#mydoclist[0:5]
count_vectorizer = CountVectorizer(stop_words='english',lowercase=True,analyzer='word',ngram_range=(1,1))
term_freq_matrix = count_vectorizer.fit_transform(mydoclist)
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(term_freq_matrix)
tf_idf_matrix = tfidf.transform(term_freq_matrix)
pos_to_word = dict([[v,k] for k,v in count_vectorizer.vocabulary_.items()])

In [20]:
n_importance = 3 #no of tags to be generated
predict = importance_list(tf_idf_matrix,n_importance)
predict_vs_actual = pd.DataFrame({'tags':predict,'id':raw['id']})
predict_vs_actual['tags'] = predict_vs_actual['tags'].apply(" ".join)

In [22]:
predict_vs_actual.to_csv("predicted.csv",index=False)

print(predict_vs_actual[0:100])


This marks the end of this submission. Unfortunately, I was not able to implement transfer learning but, I was able to generate tags for all the questions.